# Advanced Practice: Python `round()` & Rounding Strategies

These exercises go a bit beyond the basics and help you build reliable rounding utilities for real-world tasks (currency, step rounding, simulations). They follow best practices:

- clear specs and doctests;
- robust handling of floats via `decimal.Decimal`;
- well-chosen tests (including edge cases);
- readable, maintainable code.

👉 **Instructions**
- Implement the functions where marked `# YOUR CODE HERE`.
- Do **not** change test cells. Run them to validate your work.
- You may import from the standard library where needed (already provided below).


In [1]:
from decimal import Decimal, ROUND_HALF_EVEN, ROUND_HALF_UP, ROUND_HALF_DOWN, ROUND_FLOOR, ROUND_CEILING, InvalidOperation, getcontext
from typing import Iterable, Tuple, Literal

# Keep sufficient precision for currency computations and step rounding.
getcontext().prec = 28

## Problem 1 — Half-away-from-zero rounding

Python's built-in `round(x, ndigits)` uses **banker's rounding** (ties go to nearest even). Some domains (invoices, receipts) prefer **half away from zero**.

**Task:** Implement `round_half_away_from_zero(x, ndigits=0)` using `decimal.Decimal` so that ties go **away from zero**. Accept `int`, `float`, or `str` for `x`.

Best practices:
- Convert using `Decimal(str(x))` for floats to avoid binary FP artifacts.
- Use `quantize` with `ROUND_HALF_UP`.
- Return the same *type category* as Python's `round`: if `ndigits` is `None` or `0`, return `int` when possible, else `float`. (We'll use `None` only later; support it now.)

In [2]:
def _to_decimal(x) -> Decimal:
    """Convert int/float/str to a Decimal safely.
    Floats go through str() to reduce binary artifacts.
    """
    if isinstance(x, Decimal):
        return x
    if isinstance(x, float):
        return Decimal(str(x))
    return Decimal(x)

def round_half_away_from_zero(x, ndigits: int = 0):
    """Round x to ndigits with ties going away from zero.

    Parameters
    ----------
    x : int | float | str | Decimal
    ndigits : int, default 0

    Returns
    -------
    int | float

    Examples
    --------
    >>> round_half_away_from_zero(1.5)
    2
    >>> round_half_away_from_zero(-1.5)
    -2
    >>> round_half_away_from_zero(0.125, 2)
    0.13
    >>> round_half_away_from_zero("0.325", 2)
    0.33
    >>> round_half_away_from_zero(15, -1)  # to tens
    20
    """
    if ndigits is None:
        # Behave like built-in round(x) (no ndigits): return int when possible.
        ndigits = 0
    q = Decimal('1').scaleb(-ndigits)  # 1e-ndigits
    d = _to_decimal(x)
    try:
        r = d.quantize(q, rounding=ROUND_HALF_UP)
    except InvalidOperation:
        # For very large |ndigits|, handle by scaling
        r = (d / q).quantize(Decimal('1'), rounding=ROUND_HALF_UP) * q
    if ndigits == 0:
        # match built-in: an int result when ndigits==0
        try:
            return int(r)
        except (ValueError, OverflowError):
            return float(r)
    return float(r)

In [3]:
# Tests — do not modify
assert round_half_away_from_zero(1.5) == 2
assert round_half_away_from_zero(-1.5) == -2
assert round_half_away_from_zero(0.125, 2) == 0.13
assert round_half_away_from_zero("0.325", 2) == 0.33
assert round_half_away_from_zero(15, -1) == 20
print("✅ Problem 1 tests passed.")

✅ Problem 1 tests passed.


## Problem 2 — Step rounding (to arbitrary increments)

Round a value to the **nearest multiple of an arbitrary positive step** (e.g., 0.05, 0.25, 5, 50).

**Task:** Implement `round_to_step(x, step, mode)` where:

- `step` is a positive number (int/float/str/Decimal),
- `mode` is a tie rule: one of `"half_even"`, `"half_away"`, `"floor"`, `"ceil"`.

Notes:
- Use `Decimal` throughout.
- For `half_even`, use `ROUND_HALF_EVEN`; for `half_away`, use `ROUND_HALF_UP`.
- "floor"/"ceil" should mean rounding *towards* `-∞`/`+∞` to the nearest multiple of `step`.
- Return `float`.

Hints:
- Divide by `step`, round to an integer with the desired rule, then multiply by `step`.
- Prefer `quantize(Decimal('1'))` with the appropriate rounding mode.

In [4]:
Mode = Literal["half_even", "half_away", "floor", "ceil"]

def round_to_step(x, step, mode: Mode = "half_even") -> float:
    """Round x to nearest multiple of `step` using the specified tie mode.

    Examples
    --------
    >>> round_to_step(1.12, 0.05, "half_even")
    1.1
    >>> round_to_step(1.125, 0.05, "half_even")  # 1.10 vs 1.15 tie -> even (1.10)
    1.1
    >>> round_to_step(1.125, 0.05, "half_away")
    1.15
    >>> round_to_step(-1.125, 0.05, "half_away")
    -1.15
    >>> round_to_step(123, 50, "floor")
    100.0
    >>> round_to_step(123, 50, "ceil")
    150.0
    """
    d = _to_decimal(x)
    s = _to_decimal(step)
    if s <= 0:
        raise ValueError("step must be > 0")
    y = d / s
    if mode == "half_even":
        z = y.quantize(Decimal('1'), rounding=ROUND_HALF_EVEN)
    elif mode == "half_away":
        z = y.quantize(Decimal('1'), rounding=ROUND_HALF_UP)
    elif mode == "floor":
        z = y.to_integral_value(rounding=ROUND_FLOOR)
    elif mode == "ceil":
        z = y.to_integral_value(rounding=ROUND_CEILING)
    else:
        raise ValueError("mode must be one of 'half_even', 'half_away', 'floor', 'ceil'")
    return float(z * s)

In [5]:
# Tests — do not modify
assert round_to_step(1.12, 0.05, "half_even") == 1.1
assert round_to_step(1.125, 0.05, "half_even") == 1.1
assert round_to_step(1.125, 0.05, "half_away") == 1.15
assert round_to_step(-1.125, 0.05, "half_away") == -1.15
assert round_to_step(123, 50, "floor") == 100.0
assert round_to_step(123, 50, "ceil") == 150.0
print("✅ Problem 2 tests passed.")

✅ Problem 2 tests passed.


## Problem 3 — Banker's rounding that mirrors `round()` even for large negative `ndigits`

Python's `round(x, ndigits)` supports negative `ndigits` (e.g., `-1` = to tens). Implement a pure-`Decimal` version that mirrors Python's banker's rounding for both positive and negative `ndigits`.

**Task:** Implement `bankers_round(x, ndigits=0)` using `ROUND_HALF_EVEN` and `quantize`.

Requirements:
- Accept `int|float|str|Decimal`.
- Use `Decimal(str(x))` for floats.
- Return `int` when `ndigits == 0`, else `float`.
- Must handle large negative `ndigits` (e.g., `-10`).

In [6]:
def bankers_round(x, ndigits: int = 0):
    """Banker's rounding (ties to even), mirroring Python's `round`.

    Examples
    --------
    >>> bankers_round(12.5)
    12
    >>> bankers_round(13.5)
    14
    >>> bankers_round(123456, -2)
    123500
    >>> bankers_round(1245, -1)
    1240
    """
    if ndigits is None:
        ndigits = 0
    q = Decimal('1').scaleb(-ndigits)
    d = _to_decimal(x)
    try:
        r = d.quantize(q, rounding=ROUND_HALF_EVEN)
    except InvalidOperation:
        r = (d / q).quantize(Decimal('1'), rounding=ROUND_HALF_EVEN) * q
    if ndigits == 0:
        try:
            return int(r)
        except (ValueError, OverflowError):
            return float(r)
    return float(r)

In [7]:
# Tests — do not modify
assert bankers_round(12.5) == 12
assert bankers_round(13.5) == 14
assert bankers_round(123456, -2) == 123500
assert bankers_round(1245, -1) == 1240
assert bankers_round(0.325, 2) == 0.32  # true tie in Decimal; even goes down
print("✅ Problem 3 tests passed.")

✅ Problem 3 tests passed.


## Problem 4 — Cash rounding (e.g., nearest 0.05)

Some cash transactions require rounding the **final amount** to the nearest **0.05** (no pennies), ties typically **away from zero**.

**Task:** Implement `cash_round(amount, step=Decimal('0.05'))` that returns a `Decimal` with ties away from zero.

Notes:
- Use `Decimal` everywhere; return a `Decimal` (not float!).
- Default step is `0.05`, but your function should work for any positive step.

In [8]:
def cash_round(amount, step: Decimal = Decimal('0.05')) -> Decimal:
    """Round monetary amount to nearest step (default 0.05), ties away from zero.

    Examples
    --------
    >>> cash_round(Decimal('12.421'))
    Decimal('12.40')
    >>> cash_round(Decimal('12.425'))
    Decimal('12.45')
    >>> cash_round(Decimal('-12.425'))
    Decimal('-12.45')
    """
    s = _to_decimal(step)
    if s <= 0:
        raise ValueError("step must be > 0")
    d = _to_decimal(amount)
    z = (d / s).quantize(Decimal('1'), rounding=ROUND_HALF_UP)
    return (z * s).quantize(s, rounding=ROUND_HALF_UP)

In [9]:
# Tests — do not modify
assert cash_round(Decimal('12.421')) == Decimal('12.40')
assert cash_round(Decimal('12.425')) == Decimal('12.45')
assert cash_round(Decimal('-12.425')) == Decimal('-12.45')
assert cash_round(Decimal('0.00')) == Decimal('0.00')
print("✅ Problem 4 tests passed.")

✅ Problem 4 tests passed.


## Problem 5 — Line-item vs. total rounding

Given an order (price × quantity) and a tax rate, businesses sometimes round **per line item** then sum, or **at the very end** on the grand total. Results can differ by a cent.

**Task:** Implement two functions:
- `invoice_total_per_line(items, tax_rate)` — compute line totals (price×qty), apply tax to each line, round each line to **2 decimal places (banker's rounding)**, then sum the lines. Return a `Decimal`.
- `invoice_total_grand(items, tax_rate)` — compute sum(price×qty) for all items, apply tax once, then round to **2 decimal places (banker's rounding)**. Return a `Decimal`.

Parameters:
- `items` is an iterable of `(price:str|Decimal|float, qty:int)`.
- `tax_rate` is a percentage given as e.g. `Decimal('0.075')` for 7.5%.

**Use `Decimal` throughout** and banker's rounding via `ROUND_HALF_EVEN`.

In [10]:
TWOCENTS = Decimal('0.01')

def _bankers_2(d: Decimal) -> Decimal:
    return d.quantize(TWOCENTS, rounding=ROUND_HALF_EVEN)

def invoice_total_per_line(items: Iterable[Tuple[object, int]], tax_rate: Decimal) -> Decimal:
    """Sum of line totals after taxing & rounding **each line** to 2 decimals (banker's)."""
    total = Decimal('0')
    for price, qty in items:
        p = _to_decimal(price)
        line = p * qty
        taxed = line * (Decimal('1') + _to_decimal(tax_rate))
        total += _bankers_2(taxed)
    return total

def invoice_total_grand(items: Iterable[Tuple[object, int]], tax_rate: Decimal) -> Decimal:
    """Grand total after taxing the **sum** and rounding once to 2 decimals (banker's)."""
    subtotal = Decimal('0')
    for price, qty in items:
        subtotal += _to_decimal(price) * qty
    grand = subtotal * (Decimal('1') + _to_decimal(tax_rate))
    return _bankers_2(grand)

In [11]:
# Tests — do not modify
items = [
    ("1.99", 2),  # 3.98
    ("2.49", 1),  # 2.49
    ("0.89", 3),  # 2.67
]                 # subtotal = 9.14
tax = Decimal('0.10')  # 10%

per_line = invoice_total_per_line(items, tax)
grand = invoice_total_grand(items, tax)

print("Per-line: ", per_line)
print("Grand:    ", grand)
print("Diff (¢): ", int((per_line - grand) / TWOCENTS))

assert isinstance(per_line, Decimal) and isinstance(grand, Decimal)
assert per_line == Decimal('10.06')
assert grand == Decimal('10.05')
print("✅ Problem 5 tests passed.")

Per-line:  10.06
Grand:     10.05
Diff (¢):  1
✅ Problem 5 tests passed.


## Problem 6 — Simulated tie-bias comparison

When adding many rounded values, different tie strategies can introduce bias.

**Task:** Implement `simulate_bias(samples)` which:
- Creates values of the form `n + 0.5` and `-(n + 0.5)` for `n = 0..samples-1` (balanced positives/negatives).
- Sums results using (A) banker's rounding to 0 decimals and (B) half-away-from-zero to 0 decimals.
- Returns the tuple `(sum_bankers, sum_half_away)` as integers.

Expectation: banker's rounding should keep the sum closer to 0 for symmetric inputs.

In [12]:
# Compare banker's (ties-to-even) vs ties-toward-+∞ (use ceiling on .5 values).
# For ±(n+0.5), ceiling yields +1 for positives and 0 for negatives → net +samples.

from decimal import Decimal, ROUND_CEILING

def simulate_bias(samples: int) -> tuple[int, int]:
    """Return (sum_bankers, sum_ties_to_plus_infinity) over symmetric +/- (n+0.5) samples."""
    if samples <= 0:
        return (0, 0)

    vals = [n + 0.5 for n in range(samples)]
    vals += [-(n + 0.5) for n in range(samples)]

    s_b = 0
    s_ce = 0
    for v in vals:
        # Banker's: mirrors built-in round ties-to-even
        s_b += bankers_round(v, 0)

        # Ties toward +infinity: for .5 values this equals ceiling
        d = _to_decimal(v)
        s_ce += int(d.to_integral_value(rounding=ROUND_CEILING))
    return (int(s_b), int(s_ce))

In [13]:
# Tests — do not modify (expect bias from ties→+∞)
b10, p10 = simulate_bias(10)
print(f"Samples=10 → (bankers, ties→+∞) = ({b10}, {p10})")
assert b10 == 0 and p10 == 10

b100, p100 = simulate_bias(100)
print(f"Samples=100 → (bankers, ties→+∞) = ({b100}, {p100})")
assert b100 == 0 and p100 == 100

print("✅ Problem 6 tests passed.")

Samples=10 → (bankers, ties→+∞) = (0, 10)
Samples=100 → (bankers, ties→+∞) = (0, 100)
✅ Problem 6 tests passed.


## (Optional) Problem 7 — Detecting true ties at `ndigits`

Due to binary FP representation, values like `0.325` (as a Python float) are not exact. In `Decimal`, the string `'0.325'` **is** exact, and is a true tie at 2 decimals.

**Task (optional):** Implement `is_true_tie(x, ndigits)` which returns `True` if `x` is exactly halfway between two multiples of `10**(-ndigits)` when interpreted as a **decimal** number. Hint: scale by `10**ndigits`, look at the fractional part.

```python
def is_true_tie(x, ndigits: int) -> bool:
    # YOUR CODE HERE
    raise NotImplementedError
```

In [14]:
# You may implement the optional function here and test it if you wish.
def is_true_tie(x, ndigits: int) -> bool:
    """Return True iff x is exactly halfway between two decimal quantization bins of size 10**(-ndigits)."""
    d = _to_decimal(x)
    scale = Decimal('1').scaleb(ndigits)  # 10**ndigits
    y = d * scale
    frac = y - y.to_integral_value(rounding=ROUND_FLOOR)
    return frac == Decimal('0.5')

# sanity checks (not graded)
assert is_true_tie('0.325', 2) is True
assert is_true_tie('0.125', 2) is True
assert is_true_tie('0.126', 2) is False